# Predicting glioma grading of LGG or GBM using clinical, molecular and mutation features

This jupyter notebook predicts features affecting glioma grading using a Random Forest model, and then trains and evaluates multiple classification models (Logistic Regression, Random Forest, SVM) to predict glioma grade (LGG or GBM).  A summary of findings and insights in pdf format is generated.

## Google colab

You may run this notebook on Google Colab by clicking the "Open in Colab" badge below:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/OpenBioResearch/disease-focused-uci-ml-repos/blob/main/uci_glioma_grading_prediction.ipynb)


In [ ]:
# Install the ucimlrepo library to fetch the glioma dataset
%pip install ucimlrepo

# Fetch the glioma dataset from the UCI Machine Learning Repository
from ucimlrepo import fetch_ucirepo

glioma_grading_clinical_and_mutation_features = fetch_ucirepo(id=759)

X = glioma_grading_clinical_and_mutation_features.data.features
y = glioma_grading_clinical_and_mutation_features.data.targets

print(glioma_grading_clinical_and_mutation_features.metadata)
print(glioma_grading_clinical_and_mutation_features.variables)

In [ ]:
# Explore the dataset to understand its structure and content
import pandas as pd

print("Features:")
print(X.head())

print("\nTargets:")
print(y.head())

print("\nMissing values in features:")
print(X.isnull().sum())

print("\nMissing values in targets:")
print(y.isnull().sum())

for column in X.select_dtypes(include=['number']).columns:
    X[column].fillna(X[column].mean(), inplace=True)

# Replace missing values in non-numeric columns with the mode of the column
for column in X.select_dtypes(exclude=['number']).columns:
    X[column].fillna(X[column].mode()[0], inplace=True)

# Handle missing values in the target variable 'y' if it's numeric
if y.dtypes['Grade'] in ['int64', 'float64']:
    y.fillna(y.mean(), inplace=True)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape (features): {X_train.shape}")
print(f"Training set shape (target): {y_train.shape}")
print(f"Testing set shape (features): {X_test.shape}")
print(f"Testing set shape (target): {y_test.shape}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Encode categorical features
for column in X_train.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_train[column] = le.fit_transform(X_train[column])
    X_test[column] = le.transform(X_test[column])

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train.values.ravel())

importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
selected_features = X_train.columns[indices][:20]
print("Top 20 selected features:")
print(selected_features)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Define models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42)
}

# Train and evaluate models
for name, model in models.items():
    # Train the model
    model.fit(X_train[selected_features], y_train.values.ravel())

    # Make predictions
    y_pred = model.predict(X_test[selected_features])
    y_prob = model.predict_proba(X_test[selected_features])[:, 1]

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)
    report = classification_report(y_test, y_pred)

    print(f'=== {name} ===')
    print(f'Accuracy: {accuracy}')
    print(f'ROC-AUC: {roc_auc}')
    print('Classification Report:')
    print(report)
    print('\n')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F  # Add this import

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train[selected_features].values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values.ravel(), dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test[selected_features].values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values.ravel(), dtype=torch.float32).unsqueeze(1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the neural network architecture with regularization
class RegularizedNN(nn.Module):
    def __init__(self, input_size):
        super(RegularizedNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.dropout = nn.Dropout(0.5)  # Dropout layer with 50% probability
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x

# Initialize the model, loss function, and optimizer
model = RegularizedNN(input_size=len(selected_features))
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # Adding weight decay

# Training loop with early stopping
num_epochs = 50
best_val_loss = float('inf')
patience, trials = 5, 0
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_losses.append(running_loss / len(train_loader))

    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    val_loss /= len(test_loader)
    val_losses.append(val_loss)

    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_losses[-1]}, Validation Loss: {val_loss}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trials = 0
    else:
        trials += 1

    if trials >= patience:
        print(f'Early stopping on epoch {epoch+1}')
        break

model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = (outputs.numpy().squeeze() > 0.5).astype(int)
    accuracy = accuracy_score(y_test, predictions)
    roc_auc = roc_auc_score(y_test, outputs.numpy().squeeze())
    report = classification_report(y_test, predictions)

    print('=== Neural Network ===')
    print(f'Accuracy: {accuracy}')
    print(f'ROC-AUC: {roc_auc}')
    print('Classification Report:')
    print(report)


In [ ]:
import matplotlib.pyplot as plt  

# Assuming train_losses and val_losses are lists containing loss values for each epoch
def plot_loss(train_losses, val_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.legend()
    plt.show()

plot_loss(train_losses, val_losses)


In [ ]:
import numpy as np

# Function to calculate and plot feature importance
def plot_feature_importance(model, feature_names):
    # Extract the weights from the first fully connected layer
    fc1_weights = model.fc1.weight.cpu().detach().numpy()

    feature_importance = np.sum(np.abs(fc1_weights), axis=0)
    feature_importance = feature_importance / np.sum(feature_importance)

    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importance
    })

    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    plt.figure(figsize=(12, 6))
    plt.bar(importance_df['Feature'], importance_df['Importance'])
    plt.xlabel('Features')
    plt.ylabel('Importance')
    plt.title('Feature Importance')
    plt.xticks(rotation=90)
    plt.show()

feature_names = X_train[selected_features].columns
plot_feature_importance(model, feature_names)


In [ ]:
!pip install fpdf

In [ ]:
from fpdf import FPDF
import matplotlib.pyplot as plt
import os
from datetime import datetime

output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

def save_plot_as_image(fig, filename):
    filepath = os.path.join(output_dir, filename)
    fig.savefig(filepath, format='png')
    print(f"Saved plot to {filepath}")
    plt.close(fig)

def remove_non_ascii(text):
    return ''.join(i for i in text if ord(i) < 128)

pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", 'B', 16)
pdf.cell(200, 10, "Glioma Grading Clinical and Mutation Features", ln=True, align='C')
pdf.set_font("Arial", 'I', 12)
pdf.cell(200, 10, datetime.now().strftime("Date: %Y-%m-%d"), ln=True, align='C')
pdf.ln(10)
pdf.set_font("Arial", 'B', 14)
pdf.cell(200, 10, "Overview", ln=True, align='C')
pdf.ln(10)
pdf.set_font("Arial", '', 12)
overview_text = """
This report summarizes the analysis and modeling efforts undertaken to classify glioma types into Lower-Grade Glioma (LGG) and Glioblastoma Multiforme (GBM). We utilized several machine learning models, including Logistic Regression, Random Forest, Support Vector Machine (SVM), and a Neural Network implemented in PyTorch. Early stopping was employed to monitor and prevent overfitting during the training of the neural network model.

The Random Forest model was chosen for its ability to handle a large number of features and its capability to provide feature importance scores, which help in identifying the most influential features for glioma grading. The SVM and Logistic Regression models were used for their robustness and interpretability, while the neural network was employed to capture complex patterns in the data.
"""
pdf.multi_cell(0, 10, overview_text)

# Add explanation of metrics and findings
pdf.add_page()
pdf.set_font("Arial", 'B', 14)
pdf.cell(200, 10, "Evaluation Metrics and Findings", ln=True, align='C')
pdf.ln(10)
pdf.set_font("Arial", '', 12)
metrics_explanation = """
The following metrics are used to evaluate the performance of the models:

- Accuracy: The proportion of correctly classified instances among all instances. An accuracy close to 1.0 indicates a high level of correctness in the model's predictions.
- ROC-AUC: The area under the Receiver Operating Characteristic curve, indicating the model's ability to discriminate between classes. A higher ROC-AUC score indicates better performance.
- Classification Report: Includes precision, recall, and F1-score for each class. Precision is the ratio of true positive predictions to the total predicted positives, recall is the ratio of true positives to the total actual positives, and the F1-score is the harmonic mean of precision and recall.

Findings:
- The models generally achieved high accuracy, indicating good performance in classifying glioma types.
- The ROC-AUC scores were also high, suggesting that the models are effective in distinguishing between LGG and GBM.
- The classification reports provide a detailed breakdown of the performance for each class, showing high precision and recall for both LGG and GBM.
"""
pdf.multi_cell(0, 10, metrics_explanation)

# Add metrics for Logistic Regression, Random Forest, SVM, and Neural Network
metrics_summary = []
for name, model in models.items():
    model.fit(X_train[selected_features], y_train.values.ravel())
    y_pred = model.predict(X_test[selected_features])
    y_prob = model.predict_proba(X_test[selected_features])[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)
    report = classification_report(y_test, y_pred, output_dict=True)

    metrics_summary.append({
        'Model': name,
        'Accuracy': accuracy,
        'ROC-AUC': roc_auc,
        'Precision (LGG)': report['0']['precision'],
        'Recall (LGG)': report['0']['recall'],
        'F1-Score (LGG)': report['0']['f1-score'],
        'Precision (GBM)': report['1']['precision'],
        'Recall (GBM)': report['1']['recall'],
        'F1-Score (GBM)': report['1']['f1-score']
    })

pdf.set_font("Arial", 'B', 12)
pdf.cell(200, 10, "Metrics Summary Table", ln=True, align='C')
pdf.ln(10)
pdf.set_font("Arial", '', 12)

lgg_metrics = [["Model", "Accuracy", "ROC-AUC", "Precision", "Recall", "F1-Score"]]
gbm_metrics = [["Model", "Accuracy", "ROC-AUC", "Precision", "Recall", "F1-Score"]]

for metric in metrics_summary:
    lgg_metrics.append([metric['Model'], f"{metric['Accuracy']:.4f}", f"{metric['ROC-AUC']:.4f}", f"{metric['Precision (LGG)']:.4f}", f"{metric['Recall (LGG)']:.4f}", f"{metric['F1-Score (LGG)']:.4f}"])
    gbm_metrics.append([metric['Model'], f"{metric['Accuracy']:.4f}", f"{metric['ROC-AUC']:.4f}", f"{metric['Precision (GBM)']:.4f}", f"{metric['Recall (GBM)']:.4f}", f"{metric['F1-Score (GBM)']:.4f}"])

def add_table(pdf, table_data, title):
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(200, 10, title, ln=True, align='C')
    pdf.ln(5)
    pdf.set_font("Arial", 'B', 12)
    col_width = pdf.w / (len(table_data[0]) + 1)
    row_height = pdf.font_size * 1.5
    for row in table_data:
        for cell in row:
            pdf.cell(col_width, row_height, str(cell), border=1, align='C')
        pdf.ln(row_height)
    pdf.ln(10)

add_table(pdf, lgg_metrics, "LGG Metrics")
add_table(pdf, gbm_metrics, "GBM Metrics")

pdf.add_page()
pdf.set_font("Arial", 'B', 14)
pdf.cell(200, 10, "Feature Importance (Random Forest)", ln=True, align='C')
pdf.ln(10)
pdf.set_font("Arial", '', 12)
pdf.multi_cell(0, 10, "The following plot shows the importance of each feature as determined by the Random Forest model. Higher values indicate greater importance in determining glioma grading.")

# Train a Random Forest model to get feature importances
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train[selected_features], y_train.values.ravel())
rf_importances = rf_model.feature_importances_
rf_indices = np.argsort(rf_importances)[::-1]
rf_selected_features = X_train.columns[rf_indices][:20]

fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(rf_selected_features, rf_importances[rf_indices][:20])
ax.set_xlabel('Features')
ax.set_ylabel('Importance')
ax.set_title('Feature Importance (Random Forest)')
plt.xticks(rotation=90)
plt.tight_layout()
save_plot_as_image(fig, "rf_feature_importance.png")
pdf.image(os.path.join(output_dir, "rf_feature_importance.png"), x=None, y=None, w=190, h=100)

pdf.add_page()
pdf.set_font("Arial", 'B', 14)
pdf.cell(200, 10, "Training and Validation Loss", ln=True, align='C')
pdf.ln(10)
pdf.set_font("Arial", '', 12)
pdf.multi_cell(0, 10, "The following plot shows the training and validation loss over the epochs for the neural network. Monitoring these losses helps in preventing overfitting through early stopping.")

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(train_losses, label='Training Loss')
ax.plot(val_losses, label='Validation Loss')
ax.set_title('Training and Validation Loss Over Epochs')
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.legend()
save_plot_as_image(fig, "loss_plot.png")
pdf.image(os.path.join(output_dir, "loss_plot.png"), x=None, y=None, w=190, h=100)

pdf_output_path = os.path.join(output_dir, "glioma_grading_report.pdf")
pdf.output(pdf_output_path)
print(f"Report generated and saved to {pdf_output_path}")
